# 1 Introducción
El siguiente cuaderno realiza una conversion de un texto, aumentando el valor ASCII de cada caracter en forma paralela utilizando threads de un procesador GPU.

Su objetivo es codificar un texto para que no pueda ser interpretado facilmente, para luego ser enviado y ser decodificado con su funcion inversa.

---
# 2 Armado del ambiente
Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

---
# 3 Desarrollo
Ejecuta el Código CPU - GPU.

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

texto_cpu =  "Un texto es una composici\xF3n de signos codificados en un sistema de escritura que forma una unidad de sentido.Tambi\xE9n es una composici\xF3n de caracteres imprimibles (con grafema) generados por un algoritmo de cifrado que, aunque no tienen sentido para cualquier persona, s\xED puede ser descifrado por su destinatario original. En otras palabras, un texto es un entramado de signos con una intenci\xF3n comunicativa que adquiere sentido en determinado contexto.Las ideas que comunica un texto est\xE1n contenidas en lo que se suele denominar \xABmacroproposiciones\xBB, unidades estructurales de nivel superior o global, que otorgan coherencia al texto constituyendo su hilo central, el esqueleto estructural que cohesiona elementos ling\xFC\xEDsticos formales de alto nivel, como los t\xEDtulos y subt\xEDtulos, la secuencia de p\xE1rrafos, etc. En contraste, las \xABmicroproposiciones\xBB son los elementos coadyuvantes de la cohesi\xF3n de un texto, pero a nivel m\xE1s particular o local. Esta distinci\xF3n fue realizada por Teun van Dijk en 1980.1\u200BEl nivel microestructural o local est\xE1 asociado con el concepto de cohesi\xF3n. Se refiere a uno de los fen\xF3menos propios de la coherencia, el de las relaciones particulares y locales que se dan entre elementos ling\xFC\xEDsticos, tanto los que remiten unos a otros como los que tienen la funci\xF3n de conectar y organizar.Tambi\xE9n es un conjunto de oraciones agrupadas en p\xE1rrafos que habla de un tema determinado.De acuerdo a Greimas, es un enunciado ya sea gr\xE1fico o f\xF3nico que nos permite visualizar las palabras que escuchamos y que es utilizado para manifestar el proceso ling\xFC\xEDstico. Mientras Hjelmslev usa ese t\xE9rmino para designar el todo de una cadena ling\xFC\xEDstica ilimitada (\xA71).En ling\xFC\xEDstica, no todo conjunto de signos constituye un texto.Se le llama texto a la configuraci\xF3n de lengua o habla y se utilizan signos espec\xEDficos (signo de la lengua o habla) y est\xE1 organizada seg\xFAn reglas del habla o idioma.Todo texto necesariamente posee las siguientes propiedades:Cohesi\xF3n. Un texto cohesionado es aquel cuyas partes se encuentran unidas l\xF3gicamente entre s\xED, o sea, que de la lectura de una parte se puede ir a la siguiente de manera ordenada, racional. La falta de cohesi\xF3n hace que los textos salten de una cosa a otra, sin ton ni son.Coherencia. Los textos deben ser coherentes, lo cual significa centrarse en un tema o t\xF3pico sobre el cual van a referirse, sea el que sea. Un texto deber\xEDa avanzar de a poco hacia la composici\xF3n de una idea global, general, a trav\xE9s de la exposici\xF3n de ideas m\xE1s peque\xF1as o sencillas. Pero al final de la lectura de un texto coherente, uno puede explicar \u201Cde qu\xE9 trata\u201D.Significado. Todo texto posee un significado a recuperar por el lector, incluso en los m\xE1s banales o ineficientes. Pero la escritura nunca carece de significado, pues no tendr\xEDa nada que comunicar y la lectura ser\xEDa imposible.Progresividad. Un texto ofrece su contenido de manera progresiva, es decir, poco a poco, una oraci\xF3n a la vez. Por eso para saber todo lo que dice debemos leerlo todo, pues a medida que avanzamos en la lectura vamos descifrando m\xE1s y m\xE1s del contenido de su mensaje, y si nos conformamos con la primera parte, no lo sabremos todo.Intencionalidad. Todo texto es escrito con alguna intenci\xF3n comunicativa, o sea, con alg\xFAn prop\xF3sito en mente, ya sea servir de recordatorio, decirle a otra persona que haga algo, o simplemente entretener. Sea como sea, dicha intenci\xF3n configurar\xE1 el texto y har\xE1 que el emisor emplee unos u otros recursos en su composici\xF3n.Adecuaci\xF3n. Todo texto debe adaptarse a una serie de c\xF3digos y preceptos que sean comunes con su receptor, de manera que \xE9ste pueda entenderlo y descifrar su contenido. Esto pasa por el modo de uso del lenguaje, tambi\xE9n por las convenciones del g\xE9nero, etc." #@param {type: "string"}
# --------------------------------------------
try:
  from datetime import datetime
  import pycuda.driver as cuda
  import pycuda.autoinit
  from pycuda.compiler import SourceModule
  import numpy
  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos 
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------
  tiempo_inicio_total = datetime.now()
  result = numpy.empty_like(texto_cpu)
  text_size = len(texto_cpu)
  texto_cpu_array = numpy.array(texto_cpu, dtype=str)
  # CPU - reservo la memoria GPU.
  texto_gpu = cuda.mem_alloc(texto_cpu_array.size * texto_cpu_array.dtype.itemsize)
  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(texto_gpu, texto_cpu_array)
  # CPU - Defino la función kernel que ejecutará en GPU.
  module = SourceModule("""
  __global__ void kernel_codificar( char texto[], int size )
  {
    int idx = (threadIdx.x + blockIdx.x * blockDim.x) * 4;
    if(idx < size){
      texto[idx] ++;
    } 
  }
  """)
  # CPU - Genero la función kernel.
  kernel = module.get_function("kernel_codificar")
  tiempo_inicio_gpu = datetime.now()
  # Armar las dimensiones correctamente.
  dim_hilo = 256
  dim_bloque = numpy.int( (text_size + dim_hilo-1) / dim_hilo )
  # GPU - Ejecuta el kernel.
  kernel(texto_gpu, numpy.int32(text_size * 4), block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )
  tiempo_fin_gpu = datetime.now()

  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh(result, texto_gpu)
  tiempo_fin_total = datetime.now()

  print("Resultado: ", result)
  print("Cantidad de elementos: ", text_size )
  print("Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
  print("Tiempo Total: ", tiempo_en_ms(tiempo_fin_total - tiempo_inicio_total), "ms")
  print("Tiempo GPU: ", tiempo_en_ms(tiempo_fin_gpu - tiempo_inicio_gpu), "ms")
except Exception as excep:
  print("Error: ", excep)

---
# 4 Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  tiempo_en_ms          | Define una funcion para obtener el tiempo en milisegundos a partir de una fecha.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  len(texto_cpu) | Obtiene el tamaño del texto a convertir.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria texto_GPU a CPU memoria result.
CPU      |  print()               | Informo los resultados.



---
# 5 Conclusiones

Como conclusion podemos observar que al realizar la conversion en paralelo de cada caracter de un texto, usando threads del GPU, dicha conversion se realiza mucho mas rapido que si se hiciera secuencialmente. Tambien debe destacarse que la inicializacion toma bastante tiempo, por lo cual es conveniente utilizar este tipo de procesamiento cuando el texto a convertir es bastante largo.

---
# 6 Bibliografia

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[4] Repositorio de PyCUDA: [WEB](https://pypi.python.org/pypi/pycuda)